In [1]:
pip install ludwig

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 20.4 MB/s eta 0:00:00
  

In [2]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size)

    def forward(self, x):
        output, hidden = self.gru(x)
        return output, hidden

class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.gru = nn.GRU(hidden_size, output_size)

    def forward(self, x, hidden):
        output, hidden = self.gru(x, hidden)
        return output, hidden

class ECD(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout=0.1, l2_regularization=0.01):
        super(ECD, self).__init__()
        self.encoder = Encoder(input_size, hidden_size)
        self.decoder = Decoder(hidden_size, output_size)

        # Add dropout layer for regularization
        self.dropout = nn.Dropout(dropout)

        # Add L2 regularization for each parameter
        self.l2_regularization = l2_regularization

    def forward(self, inputs):
        encoder_output, encoder_hidden = self.encoder(inputs)

        # Apply dropout
        encoder_output = self.dropout(encoder_output)

        decoder_output, decoder_hidden = self.decoder(encoder_output, encoder_hidden)
        return decoder_output, decoder_hidden

    def get_loss(self):
        # Add L2 regularization term to the loss
        l2_loss = sum(torch.sum(param ** 2) for param in self.parameters())
        return l2_loss

# Example usage
input_size = 10
hidden_size = 20
output_size = 10
model = ECD(input_size, hidden_size, output_size)
print(model)


ECD(
  (encoder): Encoder(
    (gru): GRU(10, 20)
  )
  (decoder): Decoder(
    (gru): GRU(20, 10)
  )
  (dropout): Dropout(p=0.1, inplace=False)
)


In [3]:
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # 在訓練數據上進行迭代
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs, _ = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

def test_model(model, criterion):
    model.eval()
    running_loss = 0.0

    # 在測試數據上進行迭代
    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs, _ = model(inputs)
            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)

    test_loss = running_loss / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.4f}")

In [4]:
import torch
from torchvision import datasets, transforms

# 定義轉換
transform = transforms.Compose([
    transforms.ToTensor(), # 將圖像轉換為 Tensor
    transforms.Normalize((0.5,), (0.5,)) # 正規化圖像（均值為 0.5，標準差為 0.5）
])

# 下載和加載訓練數據集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# 下載和加載測試數據集
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 定義 DataLoader
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 51793605.54it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1805005.05it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 13879277.68it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7418430.21it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# 定義模型／模型架構修改
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

# 下載和加載　MNIST　數據集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# 定義模型、損失函數和優化器／正規化
model = SimpleModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練模型／擴展
def train_model(model, trainloader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 100 == 99:
                print(f"[Epoch {epoch+1}, Batch {i+1}] Loss: {running_loss / 100:.3f}")
                running_loss = 0.0

# 測試模型
def test_model(model, testloader, criterion):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy on test set: {(100 * correct / total):.2f}%")

# 訓練模型
train_model(model, trainloader, criterion, optimizer)

# 測試模型
test_model(model, testloader, criterion)


[Epoch 1, Batch 100] Loss: 0.905
[Epoch 1, Batch 200] Loss: 0.433
[Epoch 1, Batch 300] Loss: 0.393
[Epoch 1, Batch 400] Loss: 0.351
[Epoch 1, Batch 500] Loss: 0.309
[Epoch 1, Batch 600] Loss: 0.306
[Epoch 1, Batch 700] Loss: 0.272
[Epoch 1, Batch 800] Loss: 0.274
[Epoch 1, Batch 900] Loss: 0.276
[Epoch 2, Batch 100] Loss: 0.218
[Epoch 2, Batch 200] Loss: 0.229
[Epoch 2, Batch 300] Loss: 0.205
[Epoch 2, Batch 400] Loss: 0.201
[Epoch 2, Batch 500] Loss: 0.185
[Epoch 2, Batch 600] Loss: 0.201
[Epoch 2, Batch 700] Loss: 0.183
[Epoch 2, Batch 800] Loss: 0.177
[Epoch 2, Batch 900] Loss: 0.168
[Epoch 3, Batch 100] Loss: 0.163
[Epoch 3, Batch 200] Loss: 0.164
[Epoch 3, Batch 300] Loss: 0.147
[Epoch 3, Batch 400] Loss: 0.138
[Epoch 3, Batch 500] Loss: 0.132
[Epoch 3, Batch 600] Loss: 0.150
[Epoch 3, Batch 700] Loss: 0.131
[Epoch 3, Batch 800] Loss: 0.138
[Epoch 3, Batch 900] Loss: 0.132
[Epoch 4, Batch 100] Loss: 0.127
[Epoch 4, Batch 200] Loss: 0.105
[Epoch 4, Batch 300] Loss: 0.115
[Epoch 4, 